<a href="https://colab.research.google.com/github/gramagrass/estorrente/blob/main/Fine_tuning_GPT_2_Espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init Env

In [2]:
# Check that we have a GPU
!nvidia-smi

Tue Aug 17 00:13:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 --upgrade --force-reinstall -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.2 MB/s eta 0:04:27tcmalloc: large alloc 1147494400 bytes == 0x55a9fb194000 @  0x7f0694e20615 0x55a9c115b02c 0x55a9c123b17a 0x55a9c115de4d 0x55a9c124fc0d 0x55a9c11d20d8 0x55a9c11ccc35 0x55a9c115f73a 0x55a9c11d1f40 0x55a9c11ccc35 0x55a9c115f73a 0x55a9c11ce93b 0x55a9c1250a56 0x55a9c11cdfb3 0x55a9c1250a56 0x55a9c11cdfb3 0x55a9c1250a56 0x55a9c11cdfb3 0x55a9c115fb99 0x55a9c11a2e79 0x55a9c115e7b2 0x55a9c11d1e65 0x55a9c11ccc35 0x55a9c115f73a 0x55a9c11ce93b 0x55a9c11ccc35 0x55a9c115f73a 0x55a9c11cdb0e 0x55a9c115f65a 0x55a9c11cdd67 0x55a9c11ccc35
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:28tcmalloc: large alloc 1434370048 bytes == 0x55aa3f7ea000 @  0x7f0694e20615 0x55a9c115b02c 0x55a9c123b17a 0x55a9c115de4d 0x55a9c124fc0d 0x55a9c11d20d8 0x55a9c11ccc35 0x55a9c115f73a 0x55a9c11d1f40 0x55a9c11ccc35 0x55a9c115f73a 0x55a9c11ce93b 0x55a9c1250a56 0x55a9c11cdf

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorflow &> log.txt
!pip install git+https://github.com/huggingface/transformers/ > log.txt
!pip install datasets > log.txt
!pip install deepspeed==0.4.3 > log.txt
!pip install mpi4py > log.txt
import os
import sys; print('Python:',sys.version)
import torch; print('Pytorch:',torch.__version__)
import tensorflow as tf; print('Tensorflow:',tf.__version__)
import datasets; print('Datasets:',datasets.__version__)
import transformers; print('Transformers:',transformers.__version__)
import deepspeed; print('DeepSpeed:',deepspeed.__version__)

Mounted at /content/drive
  Running command git clone -q https://github.com/huggingface/transformers/ /tmp/pip-req-build-_o4j9vmv
Python: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
Pytorch: 1.7.1+cu110
Tensorflow: 2.5.0
Datasets: 1.11.0
Transformers: 4.10.0.dev0
DeepSpeed: 0.4.3


In [5]:
MODEL_NAME = 'DeepESP/gpt2-spanish'
CORPUS_PATH = 'https://raw.githubusercontent.com/gramagrass/estorrente/main/db/consolidado.txt' # AQUI PON LA URL A TU CORPUS

path_base = '/content/drive/MyDrive/FineTuningGPT2'
path_corpus = path_base + '/Corpus/'
path_dataset = path_base + '/Dataset'
path_tokenized_dataset = path_base + '/DatasetTokenized'
path_ready_dataset = path_base + '/DatasetReady'
path_model_save = path_base + '/ModelTrained'

if not os.path.exists(path_base):
  os.makedirs(path_base)
  os.makedirs(path_corpus)
  os.makedirs(path_dataset)
  os.makedirs(path_tokenized_dataset)
  os.makedirs(path_ready_dataset)
  os.makedirs(path_model_save)

!wget -P "{path_corpus}" "{CORPUS_PATH}"
!ls "{path_corpus}"

--2021-08-17 00:27:29--  https://raw.githubusercontent.com/gramagrass/estorrente/main/db/consolidado.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135659 (132K) [text/plain]
Saving to: ‘/content/drive/MyDrive/FineTuningGPT2/Corpus/consolidado.txt.8’

consolidado.txt.8   100%[===================>] 132.48K  --.-KB/s    in 0.01s   

2021-08-17 00:27:29 (11.5 MB/s) - ‘/content/drive/MyDrive/FineTuningGPT2/Corpus/consolidado.txt.8’ saved [135659/135659]

consolidado.txt    consolidado.txt.3  consolidado.txt.6
consolidado.txt.1  consolidado.txt.4  consolidado.txt.7
consolidado.txt.2  consolidado.txt.5  consolidado.txt.8


## Prepare Dataset

### Split Corpus

In [6]:
import glob
import csv
import pandas as pd
from datasets import load_dataset
file_txt = glob.glob(path_corpus+ '/*.txt')[0]
csv_file = path_dataset+'/dataset.csv'
file_txt

'/content/drive/MyDrive/FineTuningGPT2/Corpus/consolidado.txt'

In [7]:
txt = ''
df = pd.DataFrame(None, columns=["text"])
df.to_csv(csv_file, index=False, mode='w', sep='\t', header=True)

with open(file_txt) as infile:
  lines_list = infile.readlines()
  for l in lines_list:
    txt += l.replace('   ', '\n')

  txt = txt.replace(' \n', '\n').replace('\n ', '\n')  
  df = pd.DataFrame([txt], columns=["text"])
  df.to_csv(csv_file, index=False, mode='w', sep='\t', header=True)

dataset = load_dataset('csv', data_files=csv_file, split='train', delimiter='\t')
if not os.path.exists(path_dataset):
  os.makedirs(path_dataset)
dataset.save_to_disk(path_dataset)
print('Saved file in dataset')
dataset

Using custom data configuration default-9435d4283a48edf3


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9435d4283a48edf3/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.
Saved file in dataset


Dataset({
    features: ['text'],
    num_rows: 1
})

## Tokenizing

In [8]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/840k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/914 [00:00<?, ?B/s]

In [9]:
from datasets import load_from_disk

dataset = load_from_disk(path_dataset)
column_names = dataset.column_names
dataset = dataset.map(lambda x: tokenizer(x['text']),
          batch_size = 1,
          batched=True,
          remove_columns=column_names,
          writer_batch_size = 1,
)
dataset.cleanup_cache_files()
if not os.path.exists(path_tokenized_dataset):
  os.makedirs(path_tokenized_dataset)
dataset.save_to_disk(path_tokenized_dataset)
print('Tokenized file dataset')
dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

Tokenized file dataset


Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 1
})

### Group tokenized dataset

In [10]:
BLOCK_SIZE = tokenizer.max_model_input_sizes['gpt2']

def create_blocks(listIds, blockSize=1024):
  total_length = (len(listIds) // blockSize) * blockSize
  return [listIds[i:i + blockSize] for i in range(0, total_length, blockSize)]
 
def create_blocks_dataset(examples):
    result = {
      "input_ids": create_blocks(examples["input_ids"][0], BLOCK_SIZE)
    }
    result["labels"] = result["input_ids"].copy()
    return result

BLOCK_SIZE

1024

In [11]:
from datasets import load_from_disk

dataset = load_from_disk(path_tokenized_dataset)
column_names = dataset.column_names

dataset = dataset.map(
        create_blocks_dataset,
        batched=True,
        batch_size = 1,
        remove_columns=column_names,
        writer_batch_size = 1,
)
dataset = dataset.shuffle()
dataset.cleanup_cache_files()

if not os.path.exists(path_ready_dataset):
  os.makedirs(path_ready_dataset)
dataset.save_to_disk(path_ready_dataset)

print('Ready file dataset')
dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

Ready file dataset


Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 34
})

# Train: Finetuning

In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_from_disk

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [13]:
%%bash
cat <<'EOT' > ds_config.json
{
    "fp16": {
        "enabled": true,
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": true,
        "allgather_bucket_size": 2e8,
        "overlap_comm": true,
        "reduce_scatter": true,
        "reduce_bucket_size": 2e8,
        "contiguous_gradients": true,
        "cpu_offload": false
    },
  "zero_allow_untested_optimizer": true,
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 3e-4,
      "betas": [0.9, 0.999],
      "eps": 1e-6
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 3e-4,
      "warmup_num_steps": 100
    }
  },
    "gradient_accumulation_steps": "auto",
    "train_batch_size": "auto",
    "wall_clock_breakdown": false
}
EOT

In [14]:
# Load Train Dataset 
from torch.utils.data import random_split
import gc
dataset = load_from_disk(path_ready_dataset)

total_size = len(dataset)
train_size = int(0.9 * total_size)
train_dataset, val_dataset = random_split(dataset, [train_size, total_size - train_size])

for i in range(10):
  gc.collect()
  torch.cuda.empty_cache()

In [15]:
# Prepare Training
from transformers import Trainer, TrainingArguments, default_data_collator

training_args = TrainingArguments(
                                  output_dir=path_model_save+'/checkpoints',
                                  num_train_epochs=4, 
                                  logging_steps=500, 
                                  save_steps=1000,
                                  per_device_train_batch_size = 2,
                                  per_device_eval_batch_size=2,
                                  gradient_accumulation_steps = 2, 
                                  learning_rate=3e-4,
                                  adam_epsilon=1e-06,
                                  warmup_steps=100,
                                  weight_decay=0.01, 
                                  fp16=True,
                                  logging_dir='path_model_save/logs', 
                                  deepspeed='./ds_config.json')




trainer = Trainer(model=model, 
        args=training_args, 
        train_dataset=train_dataset,
        eval_dataset=val_dataset, 
        data_collator=default_data_collator)

[2021-08-17 00:28:29,973] [INFO] [distributed.py:37:init_distributed] Not using the DeepSpeed or torch.distributed launchers, attempting to detect MPI environment...
[2021-08-17 00:28:30,433] [INFO] [distributed.py:89:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=172.28.0.2, master_port=29500
[2021-08-17 00:28:30,434] [INFO] [distributed.py:47:init_distributed] Initializing torch distributed with backend: nccl


Using amp fp16 backend


In [16]:
# Train and Save
trainer.train()
model.save_pretrained(path_model_save)

[2021-08-17 00:30:26,141] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.4.3, git-hash=unknown, git-branch=unknown
[2021-08-17 00:30:26,464] [INFO] [utils.py:13:_initialize_parameter_parallel_groups] data_parallel_size: 1, parameter_parallel_size: 1
[2021-08-17 00:30:26,527] [INFO] [engine.py:180:__init__] DeepSpeed Flops Profiler Enabled: False
Using /root/.cache/torch_extensions as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/fused_adam...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/fused_adam/build.ninja...
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module fused_adam...
Time to load fused_adam op: 25.990983486175537 seconds
[2021-08-17 00:30:53,708] [INFO] [engine.py:710:_configure_optimizer] Using DeepSpeed Optimizer param name adamw as basic opt

***** Running training *****
  Num examples = 30
  Num Epochs = 4
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 28


Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...
Time to load utils op: 0.0009441375732421875 seconds
[2021-08-17 00:31:07,840] [INFO] [stage2.py:1506:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 4294967296, reducing to 4294967296


Step,Training Loss


[2021-08-17 00:31:08,300] [INFO] [stage2.py:1506:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 4294967296, reducing to 2147483648.0
[2021-08-17 00:31:08,755] [INFO] [stage2.py:1506:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 2147483648.0, reducing to 1073741824.0
[2021-08-17 00:31:09,206] [INFO] [stage2.py:1506:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 1073741824.0, reducing to 536870912.0
[2021-08-17 00:31:09,661] [INFO] [stage2.py:1506:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 536870912.0, reducing to 268435456.0
[2021-08-17 00:31:09,662] [INFO] [timer.py:160:stop] 0/10, SamplesPerSec=9.08656894095607
[2021-08-17 00:31:10,118] [INFO] [stage2.py:1506:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 268435456.0, reducing to 134217728



Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in /content/drive/MyDrive/FineTuningGPT2/ModelTrained/config.json
Model weights saved in /content/drive/MyDrive/FineTuningGPT2/ModelTrained/pytorch_model.bin


# Generar

# Instalar y Cargar Dependencias

In [17]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorflow &> log.txt
!pip install transformers > log.txt
import os
import sys; print('Python:',sys.version)
import torch; print('Pytorch:',torch.__version__)
import tensorflow as tf; print('Tensorflow:',tf.__version__)
import transformers; print('Transformers:',transformers.__version__)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Python: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
Pytorch: 1.7.1+cu110
Tensorflow: 2.5.0
Transformers: 4.10.0.dev0


# Cargar Tokenizador y Modelo

In [18]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer_es = GPT2Tokenizer.from_pretrained('DeepESP/gpt2-spanish')
model_es = GPT2LMHeadModel.from_pretrained(path_model_save)

loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/2b38e375a606e5f8f63dbc97cf8579eda0be824f6e74ed001fb5e9d11fb80774.9654cf615b595dbd278d7f33c84b3f4565133f4c3874d094bf360e7ea285dfaf
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/fb80110197f1b5c12472df03799ca4826aecf5fc97f788664c17035ea4ea004e.356a983865c1a96482df9e7e88858dbaa53473df869b0c13eb3c1d3c83a5e1fd
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/33f6ff51b9dee7e39734d410c27efdddda2751cd489da15522ac58d7a9dc32bf.0d87709f2545d45784d5ff8d35dc319de93e4087b3c680e0e20b305fb949d2a7
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/tokenizer_config.json

In [19]:
# Tokenizador
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('DeepESP/gpt2-spanish')

loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/2b38e375a606e5f8f63dbc97cf8579eda0be824f6e74ed001fb5e9d11fb80774.9654cf615b595dbd278d7f33c84b3f4565133f4c3874d094bf360e7ea285dfaf
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/fb80110197f1b5c12472df03799ca4826aecf5fc97f788664c17035ea4ea004e.356a983865c1a96482df9e7e88858dbaa53473df869b0c13eb3c1d3c83a5e1fd
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/33f6ff51b9dee7e39734d410c27efdddda2751cd489da15522ac58d7a9dc32bf.0d87709f2545d45784d5ff8d35dc319de93e4087b3c680e0e20b305fb949d2a7
loading file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/tokenizer_config.json

In [20]:
#Tokenizamos una frase
inputs = tokenizer("Es torrente y un hilo", return_tensors="pt")
inputs.input_ids

tensor([[ 1329, 24084,   324,   304, 11812]])

In [21]:
# Modelo
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('DeepESP/gpt2-spanish')
model.config

loading configuration file https://huggingface.co/DeepESP/gpt2-spanish/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/80f30f43ed5e5159d1924e70134443c131f059187f562435a148107dbf002fec.05892ce96bf9f5c8bfde40968186593b5d0feec27999103dbf7ea1d3ca7d11e1
Model config GPT2Config {
  "_name_or_path": "/content/drive/Shareddrives/Deep-ESP/Jorge-Alex/Modelos_GPT2_publicados/gpt2-spanish/",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "s

GPT2Config {
  "_name_or_path": "DeepESP/gpt2-spanish",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.10.0.dev0",
  "use_cache": true,
  "vocab_size": 50257
}

In [22]:
# Creamos un piline que une el modelo y el tokenizador
from transformers import pipeline

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
    )

In [28]:
# Generamos
generated = text_generator("El jardín se estremece por dentro ", max_length=128)
print(generated[0]['generated_text'].rsplit('.', 2)[0], '.')
#print(generated[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


El jardín se estremece por dentro ominoso, como si acabara de morir. Los perros de la cocina se han llevado a sus mascotas, que se han posado cerca, porque se han ido y sus mascotas están sin la madre y no hay ninguna. Pero las hermanas, que se han llevado también sus mascotas, se han quedado atrás en ese paraíso, con su perro, sus hermanos, y se han ido. Se han ido. Por eso nos alegra haber hecho lo mismo que nosotros .
